In [1]:
%load_ext autoreload
%autoreload 2
import torch as th
from Datasets.TinyShakespeare.prepare import prepare_to_tokenize
from Embeddings.token_embeddings import encode, decode
from TransformerModel.transformer import TransformerDecoder, TransformerTrainer
import json
import numpy as np
from itertools import chain
from Embeddings.constants import EmbedConstants
import random
import wandb
import pickle

In [2]:
text = prepare_to_tokenize("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/big_eragon.txt")
with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/vocab_big.json", "r") as file:
    vocab = json.load(file)

In [3]:
# text = encode(text, vocab)

Save encoded

In [4]:
# with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/encoded_big.pkl", "wb") as file:
#     pickle.dump(text, file)

Load from pickle

In [5]:
with open("/home/skyr/PycharmProjects/TransformerFromScratch/Datasets/Inheritance/encoded_big.pkl", "rb") as file:
    text = pickle.load(file)

In [6]:
def nearest_divisible(number, divisor):
    return number - (number % divisor)

In [7]:
num_heads = 7
embedd_dim = nearest_divisible(450,num_heads)
dropout_p = 0.3
feed_forward_size = 916
num_layers = 12
vocab_size = len(vocab)
# vocab_size = 50257
# test_lr = 1e-5
test_lr = 3e-4
block_size = 108
weight_decay = 0.01
# weight_decay = 0.007548162081615722
epochs = 1501
batch_size = 64

In [8]:
embedd_dim

448

In [9]:
wandb.init(
    project="transformer_from_scratch",
    config={
        "embedd_dim": embedd_dim,
        "num_heads": num_heads,
        "dropout_p": dropout_p,
        "feed_forward_size": feed_forward_size,
        "num_layers": num_layers,
        "vocab_size": vocab_size,
        "test_lr": test_lr,
        "block_size": block_size,
        "weight_decay": weight_decay,
        "epochs": epochs,
        "batch_size": batch_size
    }

)

wandb: Currently logged in as: tobaleo (skyrr). Use `wandb login --relogin` to force relogin


In [10]:
table = wandb.Table(columns=["predicted_text", "real_text"])

In [11]:
transformer = TransformerDecoder(embedd_dim, num_heads, dropout_p, feed_forward_size, num_layers, vocab_size,
                                     block_size)
trainer = TransformerTrainer(transformer, th.optim.AdamW, th.nn.CrossEntropyLoss, test_lr, block_size, weight_decay,vocab, table)

In [12]:
data = list(chain.from_iterable(text))

In [13]:
test = data[:int(len(data) * 0.1)]
val = data[int(len(data) * 0.1):int(len(data) * 0.2)]
train = data[int(len(data) * 0.2):]
train = th.tensor(train).long()
# val = th.tensor(val).long()
test = th.tensor(test).long()

In [14]:
# print(len(train))
trainer.train(train, test, epochs, batch_size)

Epoch: 1500 Loss: 4.170515060424805: 100%|██████████| 1501/1501 [06:08<00:00,  4.07it/s]


In [ ]:
th.save({
        "model_state_dict": transformer.state_dict(),
        "optimizer_state_dict": trainer.optimizer.state_dict(),
    }, "model.pth")
# trainer.draw_losses()

In [ ]:
input_ = encode(["The Shade was"],vocab)
input_ = th.tensor(input_).long().cuda().reshape(1, -1)
outputs = decode(transformer.generate(input_, 108, block_size), vocab)
print(''.join(outputs))

In [ ]:
# wandb.finish()